# Chapter 21: Divide and Conquer

> *"Divide and conquer — a strategy of breaking down a problem into smaller, independent subproblems, solving them, and combining the results. It is the foundation of many efficient algorithms."* — Anonymous

---

## 21.1 Introduction to Divide and Conquer

**Divide and conquer** is a fundamental algorithmic paradigm that solves a problem by:

1. **Divide:** Breaking the problem into smaller subproblems (ideally of the same type).
2. **Conquer:** Solving each subproblem recursively (or directly if trivial).
3. **Combine:** Merging the solutions of the subproblems into the solution for the original problem.

This recursive decomposition often leads to significant efficiency gains, especially when subproblems can be solved independently and combined efficiently.

### 21.1.1 Why Divide and Conquer Matters

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF DIVIDE AND CONQUER                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. EFFICIENCY: Many divide-and-conquer algorithms achieve optimal  │
│     asymptotic complexity (e.g., O(n log n) sorting, O(n log n) FFT)│
│                                                                      │
│  2. PARALLELIZATION: Independent subproblems can be solved in       │
│     parallel, making the paradigm ideal for multi-core and          │
│     distributed computing                                            │
│                                                                      │
│  3. CACHE FRIENDLINESS: Divide-and-conquer often works on           │
│     contiguous blocks of data, improving cache performance           │
│                                                                      │
│  4. FOUNDATION: Many advanced algorithms (FFT, Strassen,            │
│     closest pair) are based on divide-and-conquer                   │
│                                                                      │
│  5. MASTER THEOREM: Provides a systematic way to analyze recurrence │
│     relations, making complexity analysis straightforward            │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### 21.1.2 General Structure

```
def divide_and_conquer(problem):
    # Base case: problem is small enough
    if is_small_enough(problem):
        return solve_directly(problem)
    
    # Divide
    subproblems = divide(problem)
    
    # Conquer (solve each subproblem recursively)
    sub_solutions = [divide_and_conquer(sub) for sub in subproblems]
    
    # Combine
    return combine(sub_solutions)
```

---

## 21.2 Recurrence Relations and the Master Theorem

Divide-and-conquer algorithms are typically analyzed using recurrence relations. The **Master Theorem** provides a cookbook solution for recurrences of the form:

**T(n) = a T(n/b) + f(n)**

where:
- a ≥ 1: number of subproblems
- b > 1: factor by which input size shrinks
- f(n): cost of dividing and combining

### 21.2.1 Master Theorem Cases

1. **If f(n) = O(n^(log_b a - ε)) for some ε > 0**, then T(n) = Θ(n^(log_b a)).
2. **If f(n) = Θ(n^(log_b a) log^k n)**, then T(n) = Θ(n^(log_b a) log^(k+1) n).
3. **If f(n) = Ω(n^(log_b a + ε)) for some ε > 0, and the regularity condition a f(n/b) ≤ c f(n) for some c < 1 and large n**, then T(n) = Θ(f(n)).

### 21.2.2 Examples

| Algorithm         | Recurrence        | a | b | f(n)          | Master Theorem Case | Complexity    |
|-------------------|-------------------|---|---|----------------|---------------------|---------------|
| Binary Search     | T(n) = T(n/2) + 1 | 1 | 2 | Θ(1)           | case 2 (k=0)        | Θ(log n)      |
| Merge Sort        | T(n) = 2T(n/2) + n| 2 | 2 | Θ(n)           | case 2 (k=0)        | Θ(n log n)    |
| Strassen (naive)  | T(n) = 7T(n/2) + n²|7 | 2 | Θ(n²)          | case 1 (log₂7≈2.81)| Θ(n^2.81)     |
| Karatsuba         | T(n) = 3T(n/2) + n| 3 | 2 | Θ(n)           | case 1 (log₂3≈1.58)| Θ(n^1.58)     |

### 21.2.3 Akra–Bazzi Method

For recurrences that do not fit the Master Theorem form (e.g., uneven splits), the **Akra–Bazzi method** generalizes:

T(n) = ∑_{i=1}^k a_i T(b_i n + h_i(n)) + f(n)

Under certain conditions, the solution is:

T(n) = Θ( n^p (1 + ∫_1^n f(u)/u^(p+1) du) )

where p satisfies ∑ a_i b_i^p = 1.

---

## 21.3 Classic Example: Merge Sort (Revisited)

Merge sort divides the array into two halves, sorts each recursively, and merges them.

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result
```

**Time Complexity:** O(n log n)  
**Space Complexity:** O(n) auxiliary (not in-place).

---

## 21.4 Inversion Counting

An **inversion** in an array is a pair (i, j) such that i < j and arr[i] > arr[j]. Counting inversions measures how far the array is from sorted. It can be done in O(n log n) by augmenting merge sort.

```python
def count_inversions(arr):
    if len(arr) <= 1:
        return arr, 0
    mid = len(arr) // 2
    left, inv_left = count_inversions(arr[:mid])
    right, inv_right = count_inversions(arr[mid:])
    merged, inv_split = merge_and_count(left, right)
    return merged, inv_left + inv_right + inv_split

def merge_and_count(left, right):
    result = []
    i = j = 0
    inv_count = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
            inv_count += len(left) - i  # all remaining left elements are greater
    result.extend(left[i:])
    result.extend(right[j:])
    return result, inv_count
```

**Explanation:** When we take an element from the right array before all left elements are taken, those left elements form inversions with this right element.

---

## 21.5 Fast Fourier Transform (FFT)

The **Fast Fourier Transform** computes the Discrete Fourier Transform (DFT) in O(n log n) time, revolutionizing signal processing, polynomial multiplication, and many other fields.

### 21.5.1 Polynomial Multiplication Problem

Given two polynomials A(x) = a₀ + a₁x + ... + a_{n-1}x^{n-1} and B(x) = b₀ + b₁x + ... + b_{n-1}x^{n-1}, compute C(x) = A(x)·B(x). Naïve convolution takes O(n²). Using FFT, we can multiply in O(n log n).

### 21.5.2 Key Idea

- Represent polynomials by their values at n distinct points (the roots of unity).
- Multiply pointwise: C(ωᵏ) = A(ωᵏ)·B(ωᵏ).
- Interpolate back to coefficient form.

FFT computes the DFT (evaluation at roots of unity) recursively using divide and conquer.

### 21.5.3 Recursive Cooley–Tukey FFT

```python
import cmath

def fft(a, invert=False):
    """Cooley–Tukey FFT, a is list of complex numbers."""
    n = len(a)
    if n == 1:
        return a
    # Divide into even and odd indices
    a_even = fft(a[0::2], invert)
    a_odd = fft(a[1::2], invert)
    
    # Combine using roots of unity
    w_n = cmath.exp(2j * cmath.pi / n)
    if invert:
        w_n = 1 / w_n
    w = 1
    res = [0] * n
    for i in range(n // 2):
        res[i] = a_even[i] + w * a_odd[i]
        res[i + n // 2] = a_even[i] - w * a_odd[i]
        if invert:
            res[i] /= 2
            res[i + n // 2] /= 2
        w *= w_n
    return res

def multiply_polynomials(p, q):
    """Multiply two polynomials using FFT."""
    # Pad to power of two length
    n = 1
    while n < len(p) + len(q) - 1:
        n <<= 1
    fp = p + [0] * (n - len(p))
    fq = q + [0] * (n - len(q))
    # Convert to complex
    fp = [complex(x, 0) for x in fp]
    fq = [complex(x, 0) for x in fq]
    # FFT
    fp_fft = fft(fp)
    fq_fft = fft(fq)
    # Pointwise multiply
    for i in range(n):
        fp_fft[i] *= fq_fft[i]
    # Inverse FFT
    res = fft(fp_fft, invert=True)
    # Extract real parts (with rounding)
    return [round(res[i].real) for i in range(len(p) + len(q) - 1)]
```

**Complexity:** O(n log n)

---

## 21.6 Closest Pair of Points

Given n points in the plane, find the pair with the smallest Euclidean distance. A divide-and-conquer algorithm achieves O(n log n).

### 21.6.1 Algorithm Outline

1. **Divide:** Sort points by x-coordinate, split at median into left and right halves.
2. **Conquer:** Recursively find closest pair in left and right halves. Let δ = min(δ_left, δ_right).
3. **Combine:** Consider points within a vertical strip of width 2δ around the midline. Sort these by y-coordinate, and for each point, check distances to next few points (at most 7) within the strip.

### 21.6.2 Implementation

```python
import math

def closest_pair(points):
    """
    points: list of (x, y)
    Returns (dist, (p1, p2))
    """
    # Sort by x-coordinate for divide step
    points_sorted_x = sorted(points, key=lambda p: p[0])
    return closest_pair_rec(points_sorted_x)

def closest_pair_rec(px):
    n = len(px)
    if n <= 3:
        return brute_force(px)
    
    mid = n // 2
    mid_x = px[mid][0]
    left = px[:mid]
    right = px[mid:]
    
    d1, pair1 = closest_pair_rec(left)
    d2, pair2 = closest_pair_rec(right)
    d = min(d1, d2)
    best_pair = pair1 if d1 <= d2 else pair2
    
    # Filter points within d of midline
    strip = [p for p in px if abs(p[0] - mid_x) < d]
    # Sort by y-coordinate
    strip.sort(key=lambda p: p[1])
    
    for i in range(len(strip)):
        for j in range(i + 1, len(strip)):
            if strip[j][1] - strip[i][1] >= d:
                break
            dist = euclidean(strip[i], strip[j])
            if dist < d:
                d = dist
                best_pair = (strip[i], strip[j])
    return d, best_pair

def brute_force(points):
    min_dist = float('inf')
    best = None
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            dist = euclidean(points[i], points[j])
            if dist < min_dist:
                min_dist = dist
                best = (points[i], points[j])
    return min_dist, best

def euclidean(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])
```

**Correctness:** The strip width 2δ ensures that points farther than δ in y need not be considered, and within the strip, only a constant number of points per point need checking (proved by pigeonhole principle in a 2δ × δ rectangle).

**Complexity:** O(n log n)

---

## 21.7 Strassen's Matrix Multiplication

Naïve matrix multiplication of n×n matrices takes O(n³). Strassen's algorithm achieves O(n^log₂7) ≈ O(n^2.81) by reducing the number of recursive multiplications.

### 21.7.1 Key Insight

For 2×2 matrices, standard multiplication uses 8 multiplications. Strassen found a way using 7 multiplications (plus additions/subtractions):

```
C = A * B

Compute:
M1 = (A11 + A22) * (B11 + B22)
M2 = (A21 + A22) * B11
M3 = A11 * (B12 - B22)
M4 = A22 * (B21 - B11)
M5 = (A11 + A12) * B22
M6 = (A21 - A11) * (B11 + B12)
M7 = (A12 - A22) * (B21 + B22)

Then:
C11 = M1 + M4 - M5 + M7
C12 = M3 + M5
C21 = M2 + M4
C22 = M1 - M2 + M3 + M6
```

Recursively apply to larger matrices (padding to power of two if necessary).

### 21.7.2 Implementation (Recursive)

```python
def strassen(A, B):
    n = len(A)
    if n <= 32:  # base case threshold
        return naive_multiply(A, B)
    
    # Pad to next power of two (simplified: assume n is power of two)
    k = n // 2
    A11 = [row[:k] for row in A[:k]]
    A12 = [row[k:] for row in A[:k]]
    A21 = [row[:k] for row in A[k:]]
    A22 = [row[k:] for row in A[k:]]
    B11 = [row[:k] for row in B[:k]]
    B12 = [row[k:] for row in B[:k]]
    B21 = [row[:k] for row in B[k:]]
    B22 = [row[k:] for row in B[k:]]
    
    # Compute the 7 products
    M1 = strassen(add(A11, A22), add(B11, B22))
    M2 = strassen(add(A21, A22), B11)
    M3 = strassen(A11, sub(B12, B22))
    M4 = strassen(A22, sub(B21, B11))
    M5 = strassen(add(A11, A12), B22)
    M6 = strassen(sub(A21, A11), add(B11, B12))
    M7 = strassen(sub(A12, A22), add(B21, B22))
    
    # Combine
    C11 = add(sub(add(M1, M4), M5), M7)
    C12 = add(M3, M5)
    C21 = add(M2, M4)
    C22 = add(sub(add(M1, M3), M2), M6)
    
    # Assemble result
    C = [[0]*n for _ in range(n)]
    for i in range(k):
        for j in range(k):
            C[i][j] = C11[i][j]
            C[i][j+k] = C12[i][j]
            C[i+k][j] = C21[i][j]
            C[i+k][j+k] = C22[i][j]
    return C

def add(X, Y):
    n = len(X)
    return [[X[i][j] + Y[i][j] for j in range(n)] for i in range(n)]

def sub(X, Y):
    n = len(X)
    return [[X[i][j] - Y[i][j] for j in range(n)] for i in range(n)]
```

**Complexity:** Recurrence T(n) = 7T(n/2) + O(n²) → O(n^log₂7) ≈ O(n^2.81).

---

## 21.8 Karatsuba Algorithm for Fast Multiplication

Karatsuba multiplies two large integers (or polynomials) faster than the naïve O(n²) method, achieving O(n^log₂3) ≈ O(n^1.585).

### 21.8.1 Idea

For numbers x and y of n digits, split them:
```
x = a * 10^{n/2} + b
y = c * 10^{n/2} + d
```
Then xy = ac * 10^n + (ad + bc) * 10^{n/2} + bd.

Naïvely, this requires 4 multiplications (ac, ad, bc, bd). Karatsuba computes only 3:
- ac
- bd
- (a+b)(c+d) = ac + ad + bc + bd
Then ad+bc = (a+b)(c+d) - ac - bd.

### 21.8.2 Implementation

```python
def karatsuba(x, y):
    """Multiply two integers using Karatsuba."""
    if x < 10 or y < 10:
        return x * y
    n = max(len(str(x)), len(str(y)))
    m = n // 2
    
    # Split
    a = x // 10**m
    b = x % 10**m
    c = y // 10**m
    d = y % 10**m
    
    # Recursively compute three products
    ac = karatsuba(a, c)
    bd = karatsuba(b, d)
    ad_bc = karatsuba(a + b, c + d) - ac - bd
    
    # Combine
    return ac * 10**(2*m) + ad_bc * 10**m + bd
```

**Note:** For very large numbers, using base 2 (bits) and shifting is more efficient.

**Complexity:** Recurrence T(n) = 3T(n/2) + O(n) → O(n^log₂3).

---

## 21.9 Summary of Divide-and-Conquer Algorithms

| Algorithm           | Recurrence          | Complexity          | Notes                         |
|---------------------|---------------------|---------------------|-------------------------------|
| Merge Sort          | T(n) = 2T(n/2) + O(n)| O(n log n)          | Stable sorting                |
| Inversion Count     | T(n) = 2T(n/2) + O(n)| O(n log n)          | Augmented merge sort          |
| FFT                 | T(n) = 2T(n/2) + O(n)| O(n log n)          | Convolution, polynomial mult  |
| Closest Pair        | T(n) = 2T(n/2) + O(n)| O(n log n)          | Geometric                     |
| Strassen            | T(n) = 7T(n/2) + O(n²)| O(n^2.81)           | Matrix multiplication         |
| Karatsuba           | T(n) = 3T(n/2) + O(n)| O(n^1.585)          | Integer/polynomial multiplic. |
| Binary Search       | T(n) = T(n/2) + O(1)| O(log n)            | Searching                     |
| Exponentiation      | T(n) = T(n/2) + O(1)| O(log n)            | Fast power                    |

---

## 21.10 Practice Problems

### Problem 1: Maximum Subarray Sum (LeetCode 53)
Given an integer array, find contiguous subarray with largest sum (Kadane's algorithm is O(n); divide-and-conquer also works in O(n log n) for practice).

**Hint:** Divide array at mid; max subarray is either entirely left, entirely right, or crossing the middle.

### Problem 2: The Skyline Problem (LeetCode 218)
Given buildings (left, right, height), compute skyline outline. Use divide-and-conquer merging of skylines.

### Problem 3: Count of Smaller Numbers After Self (LeetCode 315)
Return array where result[i] = count of elements to the right of nums[i] that are smaller. Use modified merge sort.

### Problem 4: Majority Element (LeetCode 169)
Find element appearing more than n/2 times. Use divide-and-conquer: majority in left or right, then combine.

### Problem 5: K-th Smallest Element in Two Sorted Arrays (LeetCode 4)
Find median of two sorted arrays. Divide-and-conquer by eliminating halves.

### Problem 6: Polynomial Multiplication (Implement FFT)
Given two polynomials, multiply using FFT. Then compare with Karatsuba.

### Problem 7: Closest Pair of Points (LeetCode? Not exactly, but classic)
Implement the closest pair algorithm and test on random points.

### Problem 8: Strassen's Matrix Multiplication
Implement and compare with naïve multiplication for various matrix sizes.

### Problem 9: Karatsuba for Large Numbers
Implement Karatsuba and compare with built-in multiplication for very large integers.

### Problem 10: Powering a Number (LeetCode 50)
Implement pow(x, n) using divide-and-conquer (fast exponentiation).

---

## 21.11 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Chapter 4 (Divide-and-Conquer)
2. **"The Art of Computer Programming, Vol 2"** by Donald Knuth – Section 4.3.3 (Karatsuba), Section 4.6.4 (FFT)
3. **"Algorithms"** by Robert Sedgewick – Chapter 5 (Divide-and-Conquer)
4. **"Computational Geometry: Algorithms and Applications"** by de Berg et al. – Chapter 5 (Closest Pair)
5. **Original Papers**:
   - Strassen, V. (1969) – "Gaussian elimination is not optimal"
   - Cooley, J. W., & Tukey, J. W. (1965) – "An algorithm for the machine calculation of complex Fourier series"
   - Karatsuba, A., & Ofman, Y. (1962) – "Multiplication of many-digital numbers by automatic computers"
   - Bentley, J. L., & Shamos, M. I. (1976) – "Divide-and-conquer in multidimensional space"

---

> **Coming in Chapter 22**: **Greedy Algorithms** – We'll explore problems solvable by making locally optimal choices, with proofs of correctness.

---

**End of Chapter 21**